### Fashion Mnist using CNN 

Import all required libraries 

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

c:\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Read all the required data 

In [2]:
mnist = input_data.read_data_sets('../../dataSets/fashionMnist', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataSets/fashionMnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../dataSets/fashionMnist\t10k-images-idx3-ubyte.gz
Extracting ../../dataSets/fashionMnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Some python assignments so the tensorflow graph looks nice, like number of epochs, number of output classes, input vector size. 

In [3]:
image_height = 28
image_width = 28
num_input = image_height * image_width
num_classes = 10
num_epochs = 2

Placeholders for inpputs to feed in data, 

In [4]:
X_Placeholder = tf.placeholder(tf.float32, [None, num_input])
Y_Placeholder = tf.placeholder(tf.float32, [None, num_classes])

Convolution forward pass : Initialize / declare convolution layers and respective paramaters.
Begin by reshaping the imput vector to *****

As this is not optimized, lets initilaize both the convolution layers and corresponding pooling layers lookalike, with input and filters size as difference, i.e the first convolution layer being feed by reshaped input and output of first pool layer is feed to second convolution layer. Filter size for first convolution layer is set at 32 and second at 64.

Rest the hypreparams of both convolution layer are initialize with similar way. Same goes for pooling layer, input for each being there respective convolution layer output.  

In [5]:
inputReshape = tf.reshape(X_Placeholder, [-1,image_height,image_width,1])

convolutionL1 = tf.layers.conv2d(inputs=inputReshape, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
poolL1 = tf.layers.max_pooling2d(inputs=convolutionL1, pool_size=[2, 2], strides=2)

convolutionL2 = tf.layers.conv2d(inputs=poolL1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
poolL2 = tf.layers.max_pooling2d(inputs=convolutionL2, pool_size=[2, 2], strides=2)
poolFlat = tf.reshape(poolL2, [-1, 7 * 7 * 64])

hidden = tf.layers.dense(inputs=poolFlat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=hidden, rate=0.5, training=True)
output = tf.layers.dense(inputs=dropout, units=num_classes)

Finally we calculate the loss/ error from forward pass. Tensorflow will do the back prop for us(very nice of tensorflow) using Adamoptimizer(we can use other optimizers as well :)) 

In [6]:
loss = tf.losses.softmax_cross_entropy(Y_Placeholder, output)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

Calculate predictions and accuracy to know how our model flares and display same. 

In [7]:
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(Y_Placeholder, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Now we have all the ingridients to train all our algo, creat a new session and for each epoch train for all data in minibatchs of 100. 

In [10]:
m = 55000
minibatch_size = 128
num_minibatches = int(m / minibatch_size)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(num_epochs):
        for _ in range(num_minibatches):
            input_batch, labels_batch = mnist.train.next_batch(100)
            feed_dict = {X_Placeholder: input_batch, Y_Placeholder: labels_batch}
            train_step.run(feed_dict=feed_dict)

        train_accuracy = accuracy.eval(feed_dict=feed_dict)
        print("Epoch {Epoch}, training batch accuracy {Accuracy}".format(Epoch=epoch, Accuracy=train_accuracy))

    X_test = mnist.test.images 
    y_test = mnist.test.labels

    print("Testdata accuracy {CalculatedAccuracy}".format(
        CalculatedAccuracy=accuracy.eval(feed_dict={X_Placeholder: X_test, Y_Placeholder: y_test})))

Epoch 0, training batch accuracy 0.7900000214576721
Epoch 1, training batch accuracy 0.8500000238418579
Testdata accuracy 0.839900016784668
